# Tutorial: Removing Cross-Swath Bias in `ssha_karin_2`

---
**Author**: Jinbo Wang   
**Date**: 9/7/2023  
**Revison History**

---
⚠️ **Disclaimer: Preliminary** ⚠️

_This project is currently in the process of reprocessing the data. The SWOT data should not be used for scientific analyses. The code provided here is aimed at demonstrating basic techniques for data access, processing, and visualization._

### Prerequisites
Before proceeding with this tutorial, you'll need to download the SWOT KaRIn Expert product. This code is explicit on visualization. Major analysis is done by the routine `swot_ssh_utils.fit_bias`. 

In [3]:
import pylab as plt
import numpy as np
import sys
sys.path.append('../src/')
import swot_ssh_utils as swot
from cartopy import crs as ccrs
from glob import glob

def load_and_plot_ssha_data(pth, fn, lat_bounds):
    """
    Load and plot Sea Surface Height Anomaly (SSHA) data.
    
    Parameters:
    pth (str): Path to the file
    fn (str): Filename of the data
    lat_bounds (list): Latitude bounds for the data
    
    Returns:
    None
    """
    # Load data
    data = swot.SSH_L2()
    data.load_data(pth + fn, lat_bounds=lat_bounds)
    display(data.Expert)
    
    # Preprocess data
    ssha = data.Expert.ssha_karin_2
    flag = data.Expert.ancillary_surface_classification_flag
    ssha = np.where(flag == 0, ssha, np.nan)
    lon = data.Expert.longitude.values
    lat = data.Expert.latitude.values
    distance = data.Expert.cross_track_distance.values
    
    # Create plots
    fig, ax = plt.subplots(
        figsize=(10, 8),
        ncols=2,
        nrows=1,
        sharex=True,
        sharey=True,
        subplot_kw=dict(projection=ccrs.PlateCarree())
    )
    
    # Plot original data
    swot.plot_a_segment(ax[0], lon, lat, ssha, title='SSHA_KaRIn_2 original', vmin=-5, vmax=5)
    
    # Bias correction (optional)
    ssha_1 = swot.fit_bias(
        ssha, distance,
        check_bad_point_threshold=0.1,
        remove_along_track_polynomial=True
    )
    
    # mask out data in nadir and outside of 60km swath width
    distance = np.nanmean(distance, axis=0 )
    msk = (np.abs(distance) < 60e3) & (np.abs(distance) > 10e3)
    lon[:, ~msk] = np.nan
    lat[:, ~msk] = np.nan
    ssha_1[:, ~msk] = np.nan

    # Plot bias corrected data
    swot.plot_a_segment(ax[1], lon, lat, ssha_1, title='SSHA_KaRIn_2 bias corrected', vmin=-0.2, vmax=0.2)
    
    # Save and display plot
    plt.tight_layout()
    plt.savefig('../media/figures/ssha_karin_2_california.png', dpi=100)


# Change the following for your region of interest.

pass_num='013'
pth = '/mnt/flow/swot/KaRIn/SWOT_L2_LR_SSH_1.1/' #path to your data
fn = f'SWOT_L2_LR_SSH_Expert_???_{pass_num}_20230423T0422??_*_PIB0_01.nc' #filename of your data
fns=sorted(glob(pth+fn))
fn=fns[0]
# Latitude bounds
lat_bounds = [33, 44]

# Execute the function
load_and_plot_ssha_data(pth, fn, lat_bounds)

NameError: name 'glob' is not defined